In [1]:
import requests
import re
import time
import lxml.html
import openpyxl
import pandas as pd

from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
custom_header = {
    'referer' : 'http://www.riss.kr/index.do',
    #'referer' : 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=&onHanja=&strSort=&p_year1=&p_year2=&iStartCount=0&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }

# 검색키워드: 빅데이터

In [3]:
#http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0

In [4]:
paper_link_list = []
#365
for i in range(365):
    before_url = "http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&or&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

소셜 미디어 데이터 분석을 활용한 빅데이터에 대한 인식 변화 비교 분석
빅데이터와 통계학
자연재해 분석을 위한 빅데이터 마이닝 기술
웹 사이언스 : 소셜 빅데이터 마이닝 기반 이슈 분석보고서 자동 생성
바이오센싱 융합 빅데이터 컴퓨팅 아키텍처
Rhipe를 활용한 빅데이터 처리 및 분석
빅데이터 도입을 위한 중소제조공정 4M 데이터 분석
RHadoop을 이용한 빅데이터 분산처리 시스템
에너지 ICT 융합과 빅데이터 서비스
농업·농촌 디지털 전환을 위한 빅데이터 활성화 방안 연구
10
빅 데이터 분석을 위한 지지벡터기계
부산지역 교통관련 기사를 이용한 비정형 빅데이터의 정형화와 시각적 해석
사용자 의도 기반 정량적 빅데이터 시각화 가이드라인 툴
RHIPE 플랫폼에서 빅데이터 로지스틱 회귀를 위한 학습 알고리즘
빅데이터 K-평균 클러스터링을 위한 RHadoop 플랫폼
PPFP(Push and Pop Frequent Pattern Mining): 빅데이터 패턴 분석을 위한 새로운 빈발 패턴 마이닝 방법
Spark 프레임워크 기반 비정형 빅데이터 토픽 추출 시스템 설계
소셜 빅데이터를 활용한 담배 위험 예측
동적 분산병렬 하둡시스템 및 분산추론기에 응용한 서버가상화 빅데이터 플랫폼
공공 빅데이터를 이용한 UAV 위험구역검출 및 회피방법
20
빅데이터에 대한 소개와 활용사례에 관한 연구
실시간 공간 빅데이터 분석을 위한 복합 이벤트 처리 시스템 설계 및 응용
빅데이터 기반의 오피니언 마이닝 활용 연구
빅데이터의 반복적인 연산 작업을 지원하기 위한 Hadoop 기반 순환처리 시스템
빅데이터 기반의 특허 분석 시스템의 설계 및 구현
공간 HBase : 공간 빅 데이터를 위한 HBase 확장
빅 데이터를 위한 맵리듀스 프레임워크 기반의 효율적인 쿼드 트리 생성 기법
소셜 정보 추천 기법을 위한 빅 데이터 모델
웹보메트릭스 빅데이터를 활용한 암호화폐 홈페이지의 영향력과 연결망분석
세포외 기질의 구성 및 구조의 모방을 위한 바이오 빅데이터 시스템 설계
30
대중교통 빅데이터와 사

형사사법정보의 빅데이터 활용방안 연구: 구조화 범주화 관점으로
AHP를 활용한 빅데이터 역량모델 개발 연구
지방자치단체의 빅데이터 도입을 위한 제언
스포츠콘텐츠의 빅데이터 분석 활용과 전망
4차 산업혁명시대의 빅데이터 정보자원 관리전략에 관한 연구
신화와 브랜드스토리 연결고리로서의 빅데이터
XML구조를 이용한 공공 빅데이터의 선별 저장 및 시각화 방법
공간 빅 데이터를 이용한 개인 활동패턴의 시공간적 탐색: 서울시 지역의 트윗 데이터를 사례로
공공개방 데이터 활용을 위한 빅데이터 기반의 소프트웨어 아키텍처 설계에 관한 연구
제조실행시스템에의 빅데이터 적용방안에 대한 탐색적 연구
270
인공지능과 빅데이터를 통한 이미지의 존재론과 창발성
텍스트 마이닝을 기반으로 한 무용학 자료의 빅데이터 분석
공간정보 빅 데이터의 지도화와 공간적 분포 특성에 관한 연구 - 서울시 지역의 트윗 데이터를 사례로 -
로그 분석 처리율 향상을 위한 맵리듀스 기반 분할 빅데이터 분석 기법
건강정보 빅데이터와 프라이버시 침해 문제 - 내 몸의 주인은 누구인가?
미정부의 빅데이터를 위한 보안정책
보건의료 빅데이터 법제의 쟁점과 개선방향 - 시민참여형 모델구축의 탐색을 중심으로 -
공간 빅데이터 서비스 활성화를 위한 정책과제 도출
신뢰성 빅데이터 플렛폼의 연구
공공 빅데이터 플랫폼 참조모델 개발에 관한 연구
280
안전도시 구현을 위한 빅데이터 적용 알고리즘 연구
보건의료 빅데이터 활용에 관한 법·정책적 개선방안 연구
텍스트 마이닝을 기반으로 한 무용학 자료의 빅데이터 분석
눈몬: 빅데이터를 시각화한 인포그래픽 사례 비교분석 연구 -2003 미국과 캐나다에서 발생한 블랙아웃에 관한 인포그래픽 중심으로-
공간 빅데이터를 위한 동태적 시각화 모형의 개발과 적용
관객개발을 위한 공연예술 소비자 빅데이터 분석 사례 고찰
Key-value 데이터베이스를 활용한 빅데이터 처리 플랫폼의 동향 분석
공공민원 빅데이터 분석을 통한 민원통계 및 담당부서 자동분류 시스템
활용 주체별 빅데이터 수용 인식 차이에 

490
창업지원을 위한 공공기관 빅데이터 통합
빅데이터를 활용한 효율적인 중소기업 업무 처리 모델
빅데이터를 활용한 맞춤형 교육 서비스 활성화 방안연구
빅데이터 기반 개인정보 비식별화 동향
빅데이터 처리 활용 및 머신러닝 기법 적용으로 인한 도로 손상 예측 모형 개발
데이터 공학 : 하둡 기반 DW시스템 타조와 관계형 DBMS의 성능 비교
빅데이터 분석을 이용한 환경사무의 사회적 평가
빅데이터를 통해 바라본 유아 스마트미디어 교육에 대한 사회적 인식
빅데이터 군집 분석을 이용한 학습성취도 예측
빅데이터 기반 군수품 품질정보 활용방안에 대한 연구
500
데이터 공학 : 맵리듀스를 사용한 데이터 큐브의 효율적인 계산 기법
빅 데이터 텍스트 마이닝 -정치 연설을 중심으로-
빅데이터 트렌드를 이용한 섹터 투자 전략
빅데이터 환경에서 미국 커버로스 인증 적용 정책
빅데이터 토픽모델링 및 네트워크 분석을 통한문화콘텐츠학 지식구조 연구
빅데이터 관리를 위한 문서형 DB 기반 로그관리 시스템 설계
빅데이터 및 고성능컴퓨팅 프레임워크를 활용한 유전체 데이터 전처리 과정의 병렬화
DTG 빅데이터 분석 기반 지역 교통지도 확대
빅데이터에 기초한 ‘2019 개정 누리과정 평가' 키워드 네트워크 분석
공간 빅데이터를 활용한 서울시 건물부문 온실가스 배출량 공간 분석 연구
510
빅데이터를 활용한 게임 전략 및 유저 행동 패턴 분석:배틀그라운드 게임을 중심으로
빅데이터와 FDS를 활용한 보이스피싱 피해 예측 방법 연구
빅데이터 시대에 국내 온라인 신문의 인포그래픽 활용 현황 분석과 해결방안 연구
빅데이터마이닝을 이용한 회계정보처리 모형
빅데이터를 통해 바라본 청소년 활동과 역량에 대한 사회적 인식 연구
빅데이터를 활용한 ‘2017년 무주 세계태권도선수권대회' 탐색적 연구: 소셜 미디어 자료를 중심으로
네트워크 컴퓨팅 기술을 활용한 확장 가능형 빅데이터 스토리지 시스템 개발
빅데이터 환경에서 프로세스 마이닝을 이용한 영업감사 상시 모니터링 강화에 대한 연구
빅데이터 기반 정보 추천 시스

빅데이터 플랫폼과 모니터링 시스템의 융합을 이용한 BLE기반의 ZEP시스템 공격 기법에 대한 대응방안 연구
빅데이터를 활용한 산림치유 이슈 네트워크 분석 : 2005~2019년까지의 국내 뉴스 기사를 중심으로
빅데이터에 나타난 아동 비만에 대한 사회적 인식
빅데이터 분석을 이용한 세종시 건설 계획에 관한 여론 변화
빅데이터 텍스트마이닝 분석을 통한 요가에 대한 여론 분석
빅데이터 분석을 통해 본 ‘클래식 음악' : 연관 이미지, 인물 그리고 여론성향에 관한 연구
빅데이터를 통한 캠핑 트렌드 변화 연구
빅데이터를 이용한 국내 소프트웨어교육의 평판분석
빅데이터 · 사물인터넷 시대 개인정보보호법제의 발전적 전환을 위한 연구
빅데이터 기반 재난 전조감지 모델
730
빅데이터 분석을 이용한 해양 구조물 배관 자재의 소요량 예측
빅데이터 분석을 통한 국내 현대무용 관련 연구 및 뉴스보도 경향 비교분석 - 트렌드 변화 중심으로
신용카드 빅데이터를 활용한 개인사업자 창업과 폐업 분석
빅데이터 분석을 통해 본 한국 위키피디아의 지식형성 과정에 관한 연구
개인별 작업 로그 빅데이터 기반 실시간 업무성과 모니터링
빅데이터 환경에서 보안 기술에 관한 연구
데이터 공학 : 맵리듀스에서 집계 질의 스트림의 효율적인 처리 기법
빅데이터 기반의 출산율 변동 예측
하둡과 순차패턴 마이닝 기술을 통한 교통카드 빅데이터 분석
빅데이터를 통해 바라본 뇌교육과 뇌기반교육에 대한 사회적 인식
740
빅데이터 기반 실시간 불량품 발생 원인 분석 및 설비 교체주기 예측
빅데이터를 사용한 온라인 육아 커뮤니티 영유아 부모의 자녀양육 고민 분석: 딥러닝 기법의 적용
빅데이터를 활용한 사회적 이슈와 소비행동 연구
빅데이터를 활용한 전시 병력동원 응소율 분석
빅데이터 컴퓨테이션을 활용한 건축 디자인 프로세스에 관한 연구
빅데이터를 활용한 최승희 연구의 의미 재탐색
빅데이터 분석기법을 통한 정부의 산학협력 정책 트렌드 분석
빅데이터에 나타난 감성 분석
RFID/NFC 물류의 빅 데이터 처리를 위한 하둡 시스템의

비정형 빅데이터 분석을 활용한 테파마크 인식 및 마케팅 활성화 방안에 관한 연구
농업기상 빅데이터를 활용한 스마트 식물병 관리
비정형 빅데이터를 이용한 난수생성용 블록체인 오라클
대중교통 빅데이터의 다차원 네트워크 분석기법
소셜 빅데이터 분석과 기계학습을 이용한 영화흥행예측 기법의 실험적 평가
이질 공공빅데이터 기반 농장별 맞춤 병해충 예측 모델
스마트카드 빅데이터를 이용한 대중교통 수요 추정
공간 빅데이터를 활용한 농촌재생 대상지 선정 방법론 연구
뉴스/댓글 빅데이터 의미연결망 분석을 통한 옥외광고 규제 및 법령 관련 사회적 담론 연구
IIoT 미들웨어 플랫폼을 활용한 연속 제조공정의 환경센서 빅데이터 정제시스템
950
뉴스 빅데이터를 이용한 우리나라 언론의 기록관리 분야 보도 특성 분석: 1999~2018 뉴스를 중심으로
비정형 빅데이터 분석을 기반으로 한 호텔 베이커리 인식 탐색과 서비스 마케팅 촉진 방안 연구
소셜 빅데이터 분석에 의한 신 소비시장 트렌드 연구 - ‘나홀로 소비' 연관어를 중심으로 -
묻지마 범죄자들의 특성과 적절한 처우 집행을 위한 제언: 빅데이터 분석을 통한 대중적 인식을 중심으로
소셜 네트워크의 빅데이터 분석을 통한 프로야구 인식변화
국가기록원 질의로그 빅데이터 기반 이용자 정보요구 유형 분석
특집 : 빅데이터와 인문학 ; 1907년 국채보상운동의 전국적 전개양상 연구
캠핑 브랜드의 브랜드 아이덴티티(BI) 구축 및 전략 - 감성·인지적 접근을 기반으로 한 빅 데이터 및 마켓조사를 중심으로 -
산채(산나물)에 대한 소비자 의향 및 수요 분석: 소비자 의향 조사와 소셜 빅데이터 분석을 통하여
온라인 상품평 빅데이터 감성 분석 및 감성사전 확장
960
재해지도 활용성 증대를 위한 빅데이터 구축 및 적용 방안
어린이집 기관평가에 대한 언어 네트워크 분석: 언론보도 빅데이터를 중심으로
특집 2 : 개인정보 보호와 이용자 권리 ; 빅데이터 환경과 개인정보의 보호방안 -정보주체의 관점에서 바라본 비판적 검토를 중심으로-
북한 스케치 : 200

...

노인근로자의 직무특성 및 여가활동참여가 건강에 미치는 영향
Decommissioning Cost Estimation of Kori Unit 1 Using a Multi-Regression Analysis Model
독립서점의 가치적 전환에 관한 연구 - 2010년대를 중심으로
시멘틱 네트워크 분석을 활용한 생태관광 활성화 방안에 관한 연구
국가정보자원 활용을 위한 상향식 ICT 자원 및 서비스 맵 구축 방법
토픽 모델링을 이용한 인공 지능 관련 신문 기사의 보도 경향 분석
텍스트 마이닝 분석 방법론을 활용한 1인 주거공간의 융합 콘텐츠 분석
음주운전 사건시 프로야구단의 위기커뮤니케이션 전략 분석: LG트윈스 및 SK와이번스의 2019년 사건 사례를 중심으로
판소리를 활용한 소닉 브랜딩 광고 사례 분석
공공대출보상권 제도 논의를 위한 공공도서관 대출 통계 분석
2490
레그테크(RegTech)의 도입과 규제법학의 과제
서울 대도시권 대중교통체계의 통합 시간거리 접근성 산출 알고리즘 개발
초연결사회에서 개인정보보호법제 정비방안에 관한 연구
범죄징후 예측시스템에 대한 비판적 검토 - 2019년 법무부의 발표내용을 중심으로
제4차 산업혁명 기술의 기록관리 적용 방안
새로운 기술환경하에서 개인정보의 보호와 이용의 조화를 위한 개인정보보호규제 합리화 방안
MapReduce 기반 분산 이미지 특징점 추출을 활용한 빠르고 확장성 있는 이미지 검색 알고리즘
4차 산업혁명 시대의 항만물류 대응전략
온라인 관광정보의 내용 및 텍스트 네트워크
미세먼지 예측 성능 개선을 위한 CNN-LSTM 결합 방법
2500
텍스트 마이닝을 활용한 영화학 연구주제 분석
4차 산업혁명 시대에 있어 개인정보보호 관련 몇 가지 쟁점에 관한 고찰
중국의 개인정보보호 입법에 관한 연구- 사이버보안법을 중심으로
PF_Ring을 이용한 대용량 트래픽 처리 시스템
구강관리용품에 대한 검색어 분석을 통한 선호도 융합 분석 : 구글트렌드를 이용하여
ICT 생태계 변화에 따른 국내 이동통신 사업자의 대응 전략에 대한

비정형 정보와 CNN 기법을 활용한 이진 분류 모델의 고객 행태 예측
해외 글로벌 패션기업의 디지털전환 전략 동향 분석
4차 산업혁명 신기술을 기반으로 한 융복합 스마트관광 서비스브랜드
IS 핵심 지식 및 기술 평가를 통한 MIS 커리큘럼 매핑: IS 전문가를 대상으로
비콘을 이용한 개인화 추천 서비스 프레임워크
IOT 및 감성조명 콘텐츠 기반의 LED 캠핑등 디자인 개발에 관한 연구
ICT와 사회의 ‘융합'을 위한 사회적 합의 프레임워크- 인공지능과 자율주행자동차를 중심으로
기록관리 대상으로서 SNS 연구
차세대디지털도서관서비스에 대한 Y세대 이용자의 요구분석 연구
포스트코로나 시대의 언택트 교육 환경을 대비한블록체인 기반의 온라인 학습 플랫폼
2710
창의도시 브랜드 아이덴티티 시스템에 관한 연구
태양광 발전단지 내부 그리드의 안전 특성화 접지 설계에 관한 연구
현장 직무중심 디자인 역량별 요구도 분석
국내 종합대학 지적학과의 교육과정 개선방안
LeafNet: 합성곱 신경망을 이용한 식물체 분할
인공지능과 법률 서비스:현황과 과제
범죄예측 및 형사사법절차에서 알고리즘 편향성 문제와 인공지능의 활용을 위한 규범 설계
과학기술이슈에 대한 일반인의 인식분석: 토픽모델링을 활용한 원자력발전 사례
기록관리 대상으로서 SNS 연구
선수 스마트 에이전트의 법적 지위에 관한 연구
2720
비전공자를 위한 구조적 소프트웨어 교육 모형-파이썬 중심으로
핀테크 산업의 발전방향에 관한 연구
4차 산업혁명에 따른 아동과 청소년 진로코칭: 에니어그램 성격론 활용 가능성 탐색
서울시 국악분야 학교 예술강사 지원사업의 실태분석 연구
주경로 분석과 연관어 네트워크 분석을 통한 ‘구전(WoM)' 관련 연구동향 분석
STEEP-V 방법론을 활용한 미래주거예측 및 대응방안
학문 목적 한국어교육 연구 동향 분석 -토픽모델링과 언어 네트워크 분석으로-
LID 시설 관리를 위한 사물인터넷(IoT) 활용 복합 센싱 적용기술 개발
MICE산업 이해당사자의 RAISAT 인식에 관한 탐색적 연구
현장 직

토핑 모델링을 활용한 동해안 관광의 변화 분석
AI번역의 속성 및 번역주체에 대한 논의
SNS 광고에서의 개인화 요소가 광고 효과에 미치는 영향
축제의 속성으로 본 광주비엔날레 분석
벡터화 기술을 이용한 대규모 MIMO 시스템의 간단한 Toeplitz 채널 행렬 분해
어린이집 정원충족률에 따른 폐쇄위험과 수급조절 방안
SNS 기반 여론 감성 분석
산학협력과 설계 교육 중심의 창의적 공학교육 발전 방안
인공지능 시대의 윤리와 법적 과제
엣지 컴퓨팅 기반의 의료정보 시스템에 대한 현황 연구
2940
소비자 윤리 관점에서 본 인공지능 시대의 윤리교육
휴·폐경 농지 효율적 활용에 대한 연구 -성주 참외와 경산 복숭아를 중심으로-
일본종합상사의 4차 산업혁명 대응 사업에 관한 탐색적 연구
2010년-2015년 어린이집 일반특성에 따른 정원충족률 변화
CLIAM: Cloud Infrastructure Abnormal Monitoring using Machine Learning
텍스트 마이닝을 이용한 청소년 문제 토픽 모델링
철도수단이 내재된 통행사슬의 환승저항 추정방안 - 수도권 교통카드자료를 활용하여 -
사회적 네트워크 분석을 이용한 전자산업 클러스터 연구
4차 산업혁명 시대의 컴퓨터과학 전공자를 위한 NCS 교육과정 분석
선로 유지관리를 위한 국내연구 문헌분석
2950
디지털트윈 기반의 신재생에너지 서비스 플랫폼 구현방안에 대한 연구
중소기업을 위한 프라이빗 클라우드 인프라 기반 하둡 클러스터의 기본 프로토타입 설계 및 실증
디스크 기반 그래프 엔진의 입출력 성능 향상을 위한 그래프 오더링
4차 산업혁명 시대의 경찰활동  -초연결 시대의 치안정보 활용-
지능형 범죄예방 협업체계 구축에 관한 실증적 연구
텍스트 마이닝 기반 여성과학기술인 이슈 및 정책 연구
수재해 정보 플랫폼을 위한 기술트리 활용 방안 연구
ICBM 기술 특성 기반 스마트, 커넥티드 완구의 사례 분석
소프트웨어 교육의 주요 이슈와 소프트웨어 교육 발전을 위한 교육공학자의 역할
머신러닝을 활용한 클라우드

4차 산업혁명 시대, 특수교육공학의 방향
2040년 한반도 전쟁양상과 한국의 군사전략
크라우드펀딩의 부동산 산업 활용현황과 과제
미팅 테크놀로지 특성이 활용정도, 업무효율성, 직무만족도에 미치는 영향
미래 교육 콘텐츠 구성요건에 관한 근거이론연구
프롭테크(Proptech)와 가치평가: 랜드북 AI 가치평가를 사례로
한국어교육 정책에 대한 언론 보도 양상-2012년부터 2019년까지의 신문 기사를 중심으로-
인간을 고려하는 스마트그리드, 알파그리드
디바이스 유형을 고려한 온라인 멀티 채널 마케팅 효과
사용자 정보인지 향상을 위한 큐레이션 서비스 디자인에 관한 연구
3170
비대면 채널에서의 음성분석을 통한 언어폭력 유형 탐색
4차 산업혁명과 기업의 적응
경비원에 대한 한국 언론 보도 분석: 시각화분석 중심으로
4차산업혁명의 인문 디지털 콘텐츠에 대한 연구
Integrated Safety System based on IoT
중학교 수학교과서에 제시된 사회과제재의 가교성 분석
TV드라마에 나타난 선택의 스토리텔링 - 다중인격의 등장인물을 중심으로 -
조직의 오픈소스 소프트웨어 전환에 영향을 미치는 요인과 관리자의 SW 경쟁력 확보의지의 조절효과
미술품 거래정보 온라인 제공시스템 구축을 위한 정보전략계획
Integrated Safety System based on IoT
3180
4차 산업혁명과 인문학 교육의 미래
구글 트렌드를 통해 살펴 본 아시아 한류 네트워크 - EXO 사례를 중심으로
무역성과에 대한 전략적 FTA의 영향 평가: 소셜 네트워크 관점에서
대학 소프트웨어 교육의 개선 방안에 대한 연구
스마트카드자료를 활용한 지하철 승강장 동적 혼잡도 분석모형
아동보육기관의 안전 향상을 위한 IoT 기반 상생형 멀티미디어 프로토타입 연구
중소기업의 IoT 관련 정부 R&D 현황 및 정책적 시사점
영유아교육기관의 가정연계 디지털 운영관리 인터페이스 모형개발
4차 산업혁명과 기계 의료 윤리
제6회 지방선거의 온라인 참여 : 참여유형과 참여효과를 중심으로
3190
통일교육

소셜 미디어에 따른 대중음악의 수용양상 연구
핀테크 서비스에서 오프라인에서 온라인으로의 신뢰전이에 관한 연구 - 스마트뱅킹을 중심으로 -
경상북도 검시조사관의 변사사건 현장검시에 영향을 주는 요인분석
P2P 대출상환의 결정요인과 P2P 대출거래시장 활성화를 위한 제도적 요인에 관한 연구
컨테이너 터미널에 적용 가능한 인더스트리 4.0 기반 제품/서비스의 개념 설계 및 중요성 분석
온·오프라인 유통기업의 비즈니스 모델 변화에 대한 연구
헌법상 보호되는 프라이버시 개념의 변화에 관한 소고 — 독일 연방헌법재판소 판례 분석을 중심으로 —
ICT융합 에너지 신산업 해외 선진 사례 및 비즈니스 모델 분석 연구
정부운영 패러다임과 정보공개 패러다임의 상응성에 관한 연구
스마트공장에 관한 체계적 문헌 분석: 국내 학술 경향 연구
3400
외식서비스기업의 멘토링 프로그램이 조직몰입에 미치는 영향에 관한 연구
대전광역시 범죄발생의 특성과 지역 공동체내 범죄예방시스템 구축에 관한 연구
중국 신선식품 신소매 모델과 기업사례분석: 코로나19 이후를 중심으로
델파이기법을 이용한 제4차 산업혁명의 주요 기술변화 분석
공공부문 연구개발직의 도전과 기회 : 정부출연연구소 과학기술자의 현재와 미래 탐색
서울⋅경기지역 화강암의 탄성파속도와 탄성계수에 의한 암석의 일축압축강도와의 상관성 연구
샤오미 브랜드 아이덴티티 전략 연구- 애플 브랜드와 비교를 중심으로 -
제4차 산업혁명시대의 창의융합인재 양성을 위한 관광교육의 방향 연구
일반논문 : 사물인터넷시대에서 정보인권 보장을 위한 법적 고찰
납세순응 제고를 위한 납세환경 개선방안에 대한 연구
3410
어텐션 기법 및 의료 영상에의 적용에 관한 최신 동향
Web기반 ERP 커스터마이징을 위한 백엔드 개발도구의 유용성 연구
인문학 트랜스네이션 관점에서 바라본 중국수용자 분석연구: 중국 윗챗(WeChat) 맞춤형광고를 대상으로
한국의 중국 온라인 시장 진입을 위한 전략적 접근 방안에 관한 연구
국가교육위원회 설치에 대한 제20대 국회의원의 인식 분석

In [5]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d35ef44bc459dc14b7998d826d417196',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=f38d2ee1c55620dd6aae8a972f9116fb',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=2c011f08c56933ef7ecd42904f0c5d65',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=026b127801a6945e6aae8a972f9116fb',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=698d09718ef845f9c85d2949c297615a',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=2a354df682963b887ecd42904f0c5d65',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=abd467879001ba356aae8a972f9116fb',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=701f69a0f3e9d2da47de9c1710b0298d',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=abd467879001ba35e9810257f7042666',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [6]:
len(paper_link_list)

3527

In [7]:
paper_url = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url.append(tmp_url)

100%|█████████████████████████████████████████████████████████████████████████| 3527/3527 [00:00<00:00, 1768265.62it/s]


In [8]:
paper_url

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=d35ef44bc459dc14b7998d826d417196',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=f38d2ee1c55620dd6aae8a972f9116fb',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=2c011f08c56933ef7ecd42904f0c5d65',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=026b127801a6945e6aae8a972f9116fb',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=698d09718ef845f9c85d2949c297615a',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=2a354df682963b887ecd42904f0c5d65',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=abd467879001ba356aae8a972f9116fb',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=701f69a0f3e9d2da47de9c1710b0298d',
 'http://www.ris

In [9]:
len(paper_url)

3527

In [10]:
write_wb = openpyxl.Workbook()
write_ws = write_wb.active

In [11]:
"""
write_ws = write_wb.active
write_ws['A1'] = 'title_kor'
write_ws['B1'] = 'title_eng'
write_ws['C1'] = 'author'
write_ws['D1'] = 'book'
write_ws['E1'] = 'year'
write_ws['F1'] = 'keyword'
write_ws['G1'] = 'abstract_kor'
write_ws['H1'] = 'abstract_eng'
"""

"\nwrite_ws = write_wb.active\nwrite_ws['A1'] = 'title_kor'\nwrite_ws['B1'] = 'title_eng'\nwrite_ws['C1'] = 'author'\nwrite_ws['D1'] = 'book'\nwrite_ws['E1'] = 'year'\nwrite_ws['F1'] = 'keyword'\nwrite_ws['G1'] = 'abstract_kor'\nwrite_ws['H1'] = 'abstract_eng'\n"

In [12]:
write_ws = write_wb.active
write_ws['A1'] = 'title'
write_ws['B1'] = 'author'
write_ws['C1'] = 'book'
write_ws['D1'] = 'year'
write_ws['E1'] = 'keyword'
write_ws['F1'] = 'abstract_kor'
write_ws['G1'] = 'abstract_eng'

In [13]:
start = time.time()

for i, url in enumerate(paper_url):
    res = requests.get(url, headers = custom_header)
    element = lxml.html.fromstring(res.text)

    try:
        tmp_title = element.cssselect('h3.title')
        tmp_title = tmp_title[0].text_content()
        tmp_title = tmp_title.split("=")
        title_kor = tmp_title[0].strip()
        #title_eng = tmp_title[1].strip()

        tmp_author = element.cssselect('div.infoDetailL > ul > li:nth-child(1) > div > p')
        author = tmp_author[0].text_content().strip()

        tmp_book = element.cssselect('div.infoDetailL > ul > li:nth-child(3) > div > p')
        book = tmp_book[0].text_content().strip()

        tmp_year = element.cssselect('div.infoDetailL > ul > li:nth-child(5) > div > p')
        year = tmp_year[0].text_content().strip()

        tmp_keyword = element.cssselect('div.infoDetailL > ul > li:nth-child(7) > div > p')
        keyword = tmp_keyword[0].text_content().strip()

        tmp_abstract = element.cssselect('div.text.off')
        abstract_kor = tmp_abstract[0].text_content().strip()
        abstract_eng = tmp_abstract[1].text_content().strip()

        print(i, 'paper clear')

    except:
        print(i, 'ERROR')
        continue


    write_ws.append([title_kor, author, book, year, keyword, abstract_kor, abstract_eng])
    write_wb.save('./10_빅데이터.xlsx')
    
print("Running Time : ", time.time() - start)

0 paper clear
1 paper clear
2 paper clear
3 paper clear
4 paper clear
5 paper clear
6 paper clear
7 paper clear
8 paper clear
9 paper clear
10 paper clear
11 paper clear
12 paper clear
13 paper clear
14 paper clear
15 paper clear
16 paper clear
17 paper clear
18 paper clear
19 paper clear
20 paper clear
21 paper clear
22 paper clear
23 paper clear
24 paper clear
25 paper clear
26 paper clear
27 paper clear
28 paper clear
29 paper clear
30 paper clear
31 paper clear
32 paper clear
33 paper clear
34 paper clear
35 paper clear
36 paper clear
37 paper clear
38 paper clear
39 paper clear
40 paper clear
41 paper clear
42 paper clear
43 paper clear
44 paper clear
45 paper clear
46 paper clear
47 paper clear
48 paper clear
49 paper clear
50 paper clear
51 paper clear
52 paper clear
53 paper clear
54 paper clear
55 paper clear
56 paper clear
57 paper clear
58 paper clear
59 paper clear
60 paper clear
61 paper clear
62 paper clear
63 paper clear
64 paper clear
65 paper clear
66 paper clear
67 pa

519 paper clear
520 paper clear
521 paper clear
522 paper clear
523 paper clear
524 paper clear
525 paper clear
526 paper clear
527 paper clear
528 paper clear
529 paper clear
530 paper clear
531 paper clear
532 paper clear
533 paper clear
534 paper clear
535 paper clear
536 paper clear
537 paper clear
538 paper clear
539 paper clear
540 paper clear
541 paper clear
542 paper clear
543 paper clear
544 paper clear
545 paper clear
546 paper clear
547 paper clear
548 paper clear
549 paper clear
550 paper clear
551 paper clear
552 paper clear
553 paper clear
554 paper clear
555 paper clear
556 paper clear
557 paper clear
558 paper clear
559 paper clear
560 paper clear
561 paper clear
562 paper clear
563 paper clear
564 paper clear
565 paper clear
566 paper clear
567 paper clear
568 paper clear
569 paper clear
570 paper clear
571 paper clear
572 paper clear
573 paper clear
574 paper clear
575 paper clear
576 paper clear
577 paper clear
578 paper clear
579 paper clear
580 paper clear
581 pape

1034 paper clear
1035 paper clear
1036 paper clear
1037 paper clear
1038 ERROR
1039 paper clear
1040 paper clear
1041 paper clear
1042 paper clear
1043 paper clear
1044 paper clear
1045 paper clear
1046 ERROR
1047 paper clear
1048 paper clear
1049 paper clear
1050 paper clear
1051 paper clear
1052 paper clear
1053 paper clear
1054 ERROR
1055 paper clear
1056 paper clear
1057 paper clear
1058 paper clear
1059 paper clear
1060 paper clear
1061 paper clear
1062 ERROR
1063 paper clear
1064 paper clear
1065 paper clear
1066 ERROR
1067 paper clear
1068 paper clear
1069 paper clear
1070 paper clear
1071 paper clear
1072 paper clear
1073 paper clear
1074 paper clear
1075 paper clear
1076 paper clear
1077 paper clear
1078 paper clear
1079 paper clear
1080 paper clear
1081 ERROR
1082 paper clear
1083 paper clear
1084 paper clear
1085 paper clear
1086 paper clear
1087 paper clear
1088 paper clear
1089 paper clear
1090 ERROR
1091 paper clear
1092 paper clear
1093 paper clear
1094 paper clear
1095 

1539 paper clear
1540 paper clear
1541 paper clear
1542 paper clear
1543 paper clear
1544 paper clear
1545 paper clear
1546 paper clear
1547 paper clear
1548 paper clear
1549 paper clear
1550 paper clear
1551 paper clear
1552 paper clear
1553 paper clear
1554 paper clear
1555 paper clear
1556 paper clear
1557 paper clear
1558 paper clear
1559 paper clear
1560 paper clear
1561 paper clear
1562 paper clear
1563 paper clear
1564 paper clear
1565 paper clear
1566 paper clear
1567 paper clear
1568 paper clear
1569 paper clear
1570 paper clear
1571 paper clear
1572 paper clear
1573 paper clear
1574 paper clear
1575 paper clear
1576 paper clear
1577 paper clear
1578 paper clear
1579 paper clear
1580 paper clear
1581 paper clear
1582 paper clear
1583 paper clear
1584 paper clear
1585 paper clear
1586 paper clear
1587 paper clear
1588 paper clear
1589 paper clear
1590 paper clear
1591 paper clear
1592 paper clear
1593 paper clear
1594 paper clear
1595 paper clear
1596 paper clear
1597 paper cle

2022 paper clear
2023 paper clear
2024 paper clear
2025 paper clear
2026 paper clear
2027 paper clear
2028 paper clear
2029 paper clear
2030 paper clear
2031 paper clear
2032 paper clear
2033 paper clear
2034 paper clear
2035 paper clear
2036 paper clear
2037 paper clear
2038 paper clear
2039 paper clear
2040 paper clear
2041 paper clear
2042 paper clear
2043 paper clear
2044 paper clear
2045 paper clear
2046 paper clear
2047 paper clear
2048 paper clear
2049 paper clear
2050 paper clear
2051 paper clear
2052 paper clear
2053 paper clear
2054 paper clear
2055 paper clear
2056 paper clear
2057 paper clear
2058 paper clear
2059 paper clear
2060 paper clear
2061 paper clear
2062 paper clear
2063 paper clear
2064 paper clear
2065 paper clear
2066 paper clear
2067 paper clear
2068 paper clear
2069 paper clear
2070 paper clear
2071 paper clear
2072 paper clear
2073 paper clear
2074 paper clear
2075 paper clear
2076 paper clear
2077 paper clear
2078 paper clear
2079 paper clear
2080 paper cle

2505 paper clear
2506 paper clear
2507 paper clear
2508 paper clear
2509 paper clear
2510 paper clear
2511 paper clear
2512 paper clear
2513 paper clear
2514 paper clear
2515 paper clear
2516 paper clear
2517 paper clear
2518 paper clear
2519 paper clear
2520 paper clear
2521 paper clear
2522 paper clear
2523 paper clear
2524 paper clear
2525 paper clear
2526 paper clear
2527 paper clear
2528 paper clear
2529 ERROR
2530 paper clear
2531 paper clear
2532 paper clear
2533 paper clear
2534 paper clear
2535 paper clear
2536 paper clear
2537 paper clear
2538 paper clear
2539 paper clear
2540 paper clear
2541 paper clear
2542 paper clear
2543 paper clear
2544 paper clear
2545 paper clear
2546 paper clear
2547 paper clear
2548 paper clear
2549 paper clear
2550 paper clear
2551 paper clear
2552 paper clear
2553 paper clear
2554 paper clear
2555 paper clear
2556 paper clear
2557 paper clear
2558 paper clear
2559 paper clear
2560 paper clear
2561 paper clear
2562 paper clear
2563 paper clear
256

2991 paper clear
2992 paper clear
2993 paper clear
2994 paper clear
2995 paper clear
2996 paper clear
2997 paper clear
2998 paper clear
2999 paper clear
3000 paper clear
3001 paper clear
3002 paper clear
3003 paper clear
3004 paper clear
3005 paper clear
3006 paper clear
3007 paper clear
3008 paper clear
3009 paper clear
3010 paper clear
3011 paper clear
3012 paper clear
3013 paper clear
3014 paper clear
3015 paper clear
3016 paper clear
3017 paper clear
3018 paper clear
3019 ERROR
3020 paper clear
3021 paper clear
3022 paper clear
3023 paper clear
3024 paper clear
3025 paper clear
3026 paper clear
3027 paper clear
3028 paper clear
3029 paper clear
3030 paper clear
3031 paper clear
3032 paper clear
3033 paper clear
3034 paper clear
3035 paper clear
3036 paper clear
3037 paper clear
3038 paper clear
3039 paper clear
3040 paper clear
3041 paper clear
3042 paper clear
3043 paper clear
3044 paper clear
3045 paper clear
3046 paper clear
3047 paper clear
3048 paper clear
3049 paper clear
305

3474 paper clear
3475 paper clear
3476 paper clear
3477 paper clear
3478 paper clear
3479 paper clear
3480 paper clear
3481 paper clear
3482 paper clear
3483 paper clear
3484 paper clear
3485 paper clear
3486 paper clear
3487 paper clear
3488 paper clear
3489 paper clear
3490 paper clear
3491 paper clear
3492 paper clear
3493 paper clear
3494 paper clear
3495 paper clear
3496 paper clear
3497 paper clear
3498 paper clear
3499 paper clear
3500 paper clear
3501 paper clear
3502 paper clear
3503 paper clear
3504 paper clear
3505 paper clear
3506 paper clear
3507 paper clear
3508 paper clear
3509 paper clear
3510 paper clear
3511 paper clear
3512 paper clear
3513 paper clear
3514 paper clear
3515 paper clear
3516 paper clear
3517 paper clear
3518 paper clear
3519 paper clear
3520 paper clear
3521 paper clear
3522 paper clear
3523 ERROR
3524 paper clear
3525 paper clear
3526 paper clear
Running Time :  4112.049226999283
